In [ ]:
!pip install codecarbon
!pip install thop
!pip install tqdm

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR
from ptflops import get_model_complexity_info
from tqdm import tqdm
import matplotlib.pyplot as plt
import os
import time
import codecarbon

# Initialize CodeCarbon tracker
tracker = codecarbon.EmissionsTracker()
tracker.start()

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load CIFAR-10 dataset with data augmentation
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
#val_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# Load teacher model (ResNet-50)
teacher_model = models.resnet50(pretrained=True)
teacher_model.fc = nn.Linear(2048, 10)
teacher_model.load_state_dict(torch.load('teacher_model_best_resnet.pth', map_location=device), strict=False)
teacher_model.to(device)
teacher_model.eval()

# Load student model (ResNet-18)
student_model = models.resnet18(pretrained=True)
student_model.fc = nn.Linear(512, 10)
student_model.to(device)

# Define Dynamic Distillation Loss
class DynamicDistillationLoss(nn.Module):
    def __init__(self, initial_T=5.0, final_T=2.0, alpha=0.7, total_epochs=40):
        super(DynamicDistillationLoss, self).__init__()
        self.initial_T = initial_T
        self.final_T = final_T
        self.alpha = alpha
        self.total_epochs = total_epochs
        self.criterion_ce = nn.CrossEntropyLoss()

    def forward(self, student_logits, teacher_logits, labels, current_epoch):
        T = self.initial_T - (self.initial_T - self.final_T) * (current_epoch / self.total_epochs)
        distillation_loss = F.kl_div(
            F.log_softmax(student_logits / T, dim=1),
            F.softmax(teacher_logits / T, dim=1),
            reduction='batchmean'
        ) * (T ** 2)
        student_loss = self.criterion_ce(student_logits, labels)
        return self.alpha * distillation_loss + (1 - self.alpha) * student_loss

criterion = DynamicDistillationLoss()
optimizer_student = optim.Adam(student_model.parameters(), lr=0.0005, weight_decay=1e-4)
scheduler = StepLR(optimizer_student, step_size=10, gamma=0.5)

# Function to calculate FLOPs
def calculate_flops(model):
    model.eval()
    flops, _ = get_model_complexity_info(model, (3, 32, 32), as_strings=False, print_per_layer_stat=False)
    return flops

# Count non-zero parameters
def count_non_zero_params(model):
    non_zero_count = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return non_zero_count

# Measure inference time
def measure_inference_time(model, dataloader):
    model.eval()
    start_time = time.time()
    with torch.no_grad():
        for inputs, _ in dataloader:
            inputs = inputs.to(device)
            _ = model(inputs)
    end_time = time.time()
    return (end_time - start_time) / len(dataloader)

# Training and evaluation loop
def train_and_evaluate(student_model, teacher_model, train_loader, val_loader, num_epochs):
    train_start_time = time.time()
    best_val_accuracy = 0.0
    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []

    for epoch in range(num_epochs):
        student_model.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0

        for inputs, labels in tqdm(train_loader, desc=f"Training Epoch {epoch + 1}/{num_epochs}"):
            inputs, labels = inputs.to(device), labels.to(device)
            student_outputs = student_model(inputs)
            with torch.no_grad():
                teacher_outputs = teacher_model(inputs)

            loss = criterion(student_outputs, teacher_outputs, labels, epoch)
            optimizer_student.zero_grad()
            loss.backward()
            optimizer_student.step()

            running_loss += loss.item()
            _, predicted = torch.max(student_outputs, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

        train_accuracy = 100 * correct_train / total_train
        train_losses.append(running_loss / len(train_loader))
        train_accuracies.append(train_accuracy)

        # Validation phase
        student_model.eval()
        val_loss = 0.0
        correct_val = 0
        total_val = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                student_outputs = student_model(inputs)
                teacher_outputs = teacher_model(inputs)

                loss = criterion(student_outputs, teacher_outputs, labels, epoch)
                val_loss += loss.item()
                _, predicted = torch.max(student_outputs, 1)
                total_val += labels.size(0)
                correct_val += (predicted == labels).sum().item()

        val_accuracy = 100 * correct_val / total_val
        val_losses.append(val_loss / len(val_loader))
        val_accuracies.append(val_accuracy)
        scheduler.step()

        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            torch.save(student_model.state_dict(), 'best_student_model.pth')

    train_end_time = time.time()
    training_time = (train_end_time - train_start_time) / 60

    return train_losses, train_accuracies, val_losses, val_accuracies, training_time

# Train the model
train_losses, train_accuracies, val_losses, val_accuracies, training_time = train_and_evaluate(
    student_model, teacher_model, train_loader, test_loader, num_epochs=40
)

# Model Analysis
student_model.load_state_dict(torch.load('best_student_model.pth'))
flops = calculate_flops(student_model)
param_count = count_non_zero_params(student_model)
inference_time = measure_inference_time(student_model, test_loader)
model_size = os.path.getsize('best_student_model.pth') / (1024 * 1024)
emissions = tracker.stop()

print("\n--- Model Analysis ---")
print(f"Parameter Count: {param_count}")
print(f"Model Size: {model_size:.2f} MB")
print(f"FLOPs: {flops / 1e9:.2f} GFLOPs")
print(f"Training Time: {training_time:.2f} minutes")
print(f"Average Inference Time: {inference_time:.6f} seconds")
print("\n--- Energy and Emissions Report ---")
print(f"CO2 Emissions: {emissions:.6f} kg")


[codecarbon INFO @ 01:13:19] [setup] RAM Tracking...
[codecarbon INFO @ 01:13:19] [setup] GPU Tracking...
[codecarbon INFO @ 01:13:19] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 01:13:19] [setup] CPU Tracking...
[codecarbon WARNING @ 01:13:19] No CPU tracking mode found. Falling back on CPU constant mode. 
 Linux OS detected: Please ensure RAPL files exist at \sys\class\powercap\intel-rapl to measure CPU

[codecarbon WARNING @ 01:13:20] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon INFO @ 01:13:20] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 01:13:20] >>> Tracker's metadata:
[codecarbon INFO @ 01:13:20]   Platform system: Linux-6.1.85+-x86_64-with-glibc2.35
[codecarbon INFO @ 01:13:20]   Python version: 3.10.12
[codecarbon INFO @ 01:13:20]   CodeCarbon version: 2.7.4
[codecarbon INFO @ 01:13:20]   Available RAM : 12.675 GB
[codecarbon INFO @ 01:13:20]   CPU count: 2
[codeca

Using device: cuda
Files already downloaded and verified
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
<ipython-input-2-6051aa6d8913>:46: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more detail


--- Model Analysis ---
Parameter Count: 11181642
Model Size: 42.73 MB
FLOPs: 0.04 GFLOPs
Training Time: 58.45 minutes
Average Inference Time: 0.032430 seconds

--- Energy and Emissions Report ---
CO2 Emissions: 0.040562 kg


/usr/local/lib/python3.10/dist-packages/codecarbon/output_methods/file.py:52: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame.from_records([dict(total.values)])])


In [ ]:
# Evaluate on the test dataset
student_model.eval()
correct_test, total_test = 0, 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = student_model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        correct_test += (predicted == labels).sum().item()
        total_test += labels.size(0)

test_accuracy = 100 * correct_test / total_test
print(f"Test Accuracy: {test_accuracy:.2f}%")


Test Accuracy: 87.40%


In [ ]:
pip install ptflops